# Helmholtz 1D - Mock Cycle 2-level Predictions

## Goal
For the 1D Helhmholtz operator $A = -\Delta + k^2 I$, compare the quality of SVD coarsening and pointwise coarsening by the mock cycle asymptotic convergence factor. "pt" donotes pointwise coarsening.

## Discretization
* Fixed periodic domain with $n=16$ points and different $kh$ values..
* 3-point finite difference $A^h = [1, -2 + (kh)^2, 1]$. 
* Kaczmarz relaxation. For comparison, for $kh=0$ only we also look at Gauss-Seidel in lexicographic ordering.

In [11]:
import logging
import numpy as np
import helmholtz as hm
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics.pairwise
import sys
from numpy.ma.testutils import assert_array_almost_equal
from numpy.linalg import eig, norm

%load_ext autoreload
%autoreload 2

np.set_printoptions(linewidth=500, precision=2, suppress=False)
for handler in logging.root.handlers[:]: logging.root.removeHandler(handler)
logging.basicConfig(stream=sys.stdout, level=logging.WARN, format="%(levelname)-8s %(message)s",
                    datefmt="%a, %d %b %Y %H:%M:%S")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Mock Cycle Convergence Factor

In [12]:
def create_svd_coarsening(level):
    # Generate relaxed test matrix.
    n = level.a.shape[0]
    x = hm.run.random_test_matrix((n,))
    lam = 0
    b = np.zeros_like(x)
    x, _, _ = hm.run.run_iterative_method(level.operator, lambda x, lam: (level.relax(x, b, lam), lam),
                                          x, lam, num_sweeps=100)
    # Generate coarse variables (R) based on a window of x.
    aggregate_size = 4
    x_aggregate_t = x[:aggregate_size].transpose()
    r, s = hm.coarsening.create_coarsening(x_aggregate_t, 0.1)
    #print(s, r.asarray().shape)
    #print(r.asarray())

    # Convert to sparse matrix + tile over domain.
    r_csr = r.tile(n // aggregate_size)
    return r_csr

def create_pointwise_coarsening(level):
    aggregate_size = 2
    r = hm.coarsening.Coarsener(np.array([[1, 0]]))
    # Convert to sparse matrix + tile over domain.
    domain_size = level.a.shape[0]
    r_csr = r.tile(domain_size // aggregate_size)
    return r_csr

def create_average_coarsening(level):
    aggregate_size = 2
    r = hm.coarsening.Coarsener(np.array([[1, 1]]))
    # Convert to sparse matrix + tile over domain.
    domain_size = level.a.shape[0]
    r_csr = r.tile(domain_size // aggregate_size)
    return r_csr

def mock_cycle_conv_factor(n, kh, relax, coarsening, num_relax_sweeps):
    a = hm.linalg.helmholtz_1d_operator(kh, n)
    level = hm.multilevel.Level.create_finest_level(a)

    if relax == "kaczmarz":
        relaxer = lambda x, b: level.relax(x, b, lam=0)
    elif relax == "gs":
        gs_relaxer = hm.relax.GsRelaxer(a)
        relaxer = lambda x, b: gs_relaxer.step(x, b)
    else:
        raise Exception("Unsupported relaxation type {}".format(relax))

    if coarsening == "svd":
        r = create_svd_coarsening(level)
    elif coarsening == "pt":
        r = create_pointwise_coarsening(level)
    elif coarsening == "avg":
        r = create_average_coarsening(level)
#        print(r.shape)
#        print(r.todense())
    else:
        raise Exception("Unsupported coarsening type {}".format(coarsening))

    mock_cycle = hm.mock_cycle.MockCycle(relaxer, r, num_relax_sweeps)
    x = hm.run.random_test_matrix((n,), num_examples=1)
    lam = 0
    x, _, conv_factor = hm.run.run_iterative_method(
        level.operator, lambda x, lam: (mock_cycle(x), lam), x, lam, num_sweeps=10)
    return conv_factor

In [13]:
kh_values = np.linspace(0, 1, num=11)
nu_values = range(1, 5)
coarsening_values = ("svd", "pt", "avg")
n = 16

def conv_factor(n, coarsening):
    gs = np.array([[mock_cycle_conv_factor(n, 0, "gs", coarsening, nu) for nu in nu_values]])
    conv_factor = np.array([[mock_cycle_conv_factor(n, kh, "kaczmarz", coarsening, nu) 
                             for nu in nu_values] for kh in kh_values])
    return np.concatenate((gs, conv_factor))

def conv_factor_table(n):
    result = np.concatenate(
        tuple(conv_factor(n, coarsening) for coarsening in coarsening_values),
        axis=1)
    return pd.DataFrame(result, 
                          index=("0, GS", ) + 
                          tuple(("{:.1f}".format(kh) for kh in kh_values)),
                         columns=tuple("{}, nu={}".format(coarsening, nu) for coarsening in coarsening_values
                                      for nu in nu_values))

### n = 16

In [14]:
conv_factor_table(16)

,"svd, nu=1","svd, nu=2","svd, nu=3","svd, nu=4","pt, nu=1","pt, nu=2","pt, nu=3","pt, nu=4","avg, nu=1","avg, nu=2","avg, nu=3","avg, nu=4"
"0, GS",0.293357,0.108419,0.046703,0.014920,0.656356,0.629773,0.584264,0.580963,0.290274,0.125367,0.061037,0.027479
0.0,0.440269,0.234479,0.139997,0.102781,0.494566,0.560503,0.482224,0.480922,0.393169,0.190130,0.178033,0.136981
0.1,0.387282,0.223653,0.138032,0.122998,0.460723,0.538848,0.472311,0.480932,0.442001,0.188485,0.179037,0.133711
0.2,0.447983,0.241054,0.139811,0.119573,0.501487,0.499791,0.494074,0.496867,0.385747,0.186255,0.177636,0.122391
0.3,0.440285,0.225083,0.139564,0.117275,0.510479,0.553412,0.490820,0.492830,0.458037,0.155749,0.162431,0.149682
0.4,0.464637,0.242730,0.141387,0.127179,0.504181,0.544863,0.494860,0.485485,0.362512,0.198897,0.194467,0.156119
0.5,0.469691,0.240139,0.132871,0.140123,0.511935,0.540763,0.529133,0.473880,0.437243,0.198044,0.200951,0.152921
0.6,0.281638,0.191242,0.073257,0.047793,0.549999,0.537122,0.515308,0.480343,0.493628,0.213771,0.209496,0.177888
0.7,0.483068,0.240303,0.105519,0.132708,0.580044,0.507114,0.532493,0.488531,0.443598,0.213636,0.213798,0.171763
0.8,0.535325,0.245256,0.125773,0.109461,0.627609,0.459305,0.528652,0.503517,0.592555,0.289429,0.218972,0.216663


### n = 32

In [17]:
result = conv_factor_table(32)
result

,"svd, nu=1","svd, nu=2","svd, nu=3","svd, nu=4","pt, nu=1","pt, nu=2","pt, nu=3","pt, nu=4","avg, nu=1","avg, nu=2","avg, nu=3","avg, nu=4"
"0, GS",0.293060,0.103848,0.064797,0.017064,0.655185,0.621926,0.580444,0.569006,0.307565,0.130233,0.064865,0.023297
0.0,0.416041,0.218472,0.143861,0.128018,0.490804,0.573147,0.469700,0.484409,0.391562,0.207971,0.184884,0.144566
0.1,0.429931,0.214770,0.159889,0.129624,0.469447,0.574482,0.508544,0.511213,0.457347,0.193309,0.182382,0.154271
0.2,0.471455,0.210919,0.152416,0.125545,0.501410,0.522220,0.499133,0.489463,0.471607,0.201124,0.184581,0.154678
0.3,0.421573,0.219865,0.162567,0.124853,0.519921,0.567344,0.499572,0.488575,0.504586,0.213486,0.195898,0.166173
0.4,0.489103,0.238725,0.147301,0.137286,0.503383,0.554509,0.510262,0.479946,0.456229,0.205439,0.202415,0.177729
0.5,0.486759,0.255914,0.147146,0.132515,0.552477,0.544223,0.485089,0.480096,0.494787,0.206530,0.186839,0.203023
0.6,0.528919,0.264354,0.152280,0.123336,0.572613,0.514533,0.539462,0.471824,0.517059,0.224483,0.222340,0.216703
0.7,0.589188,0.280078,0.143409,0.125649,0.601206,0.494316,0.539033,0.473147,0.488220,0.221889,0.227746,0.225177
0.8,0.581069,0.303209,0.154592,0.111488,0.629119,0.433460,0.519302,0.508860,0.614858,0.298355,0.225758,0.255526


## Conclusions
* SVD coarsening exhibits better convergence rates than pointwise. The rate improves with $nu$, while for pointwise it is bounded by $0.5$.
* Gauss-Seidel is a better smoother for the Poisson case (as it well known).
* Almost the same efficiency is maintained for all values of $0 \leq kh \leq 1$.
* The result is independent of $n$.

### Question
For $kh = 0$ and Gauss-Seidel, the predicted 2-level convergence factor does not improve with $\nu$ and does not get below $\sim 0.5$, even though in practice we know that it does and that pointwise coarsening is a perfectly good coarsening of the Laplace operator.
* Why are we not getting a quantitative prediction with the mock cycle? (We could compare with local mode analysis to gain an insight into the slowest-to-converge component.)
* Can we trust it for other $kh$ values?

In [16]:
print(result.to_latex(float_format=lambda x: '%.2f' % x))

\begin{tabular}{lrrrrrrrrrrrr}
\toprule
{} &  svd, nu=1 &  svd, nu=2 &  svd, nu=3 &  svd, nu=4 &  pt, nu=1 &  pt, nu=2 &  pt, nu=3 &  pt, nu=4 &  avg, nu=1 &  avg, nu=2 &  avg, nu=3 &  avg, nu=4 \\
\midrule
0, GS &       0.30 &       0.11 &       0.07 &       0.02 &      0.65 &      0.60 &      0.57 &      0.54 &       0.32 &       0.12 &       0.07 &       0.02 \\
0.0   &       0.42 &       0.22 &       0.15 &       0.12 &      0.49 &      0.55 &      0.49 &      0.50 &       0.45 &       0.21 &       0.19 &       0.15 \\
0.1   &       0.46 &       0.22 &       0.15 &       0.13 &      0.50 &      0.55 &      0.48 &      0.48 &       0.42 &       0.20 &       0.19 &       0.15 \\
0.2   &       0.45 &       0.24 &       0.15 &       0.12 &      0.51 &      0.58 &      0.50 &      0.48 &       0.45 &       0.21 &       0.19 &       0.16 \\
0.3   &       0.46 &       0.23 &       0.15 &       0.12 &      0.49 &      0.56 &      0.50 &      0.49 &       0.42 &       0.20 &       0.20 &   